# **Auxilliary**

In [15]:
# Library Imports
import sys
import numpy as np
import pandas as pd

from dataset import dataset_generator
from bot.bot import Bot
from modelling import data_preparation as dp
from modelling import model_trainer as mt

path = "binary-options-research/"
if path not in sys.path:
    sys.path.append(path)
sys.stdout = open("stdout", 'w')

np.random.seed(42)

%load_ext autoreload
%autoreload 2

# **Data Generation**

In [5]:
# Dataset Generation
dataset_generator.make(["BTCUSDT"], "15m")

# **Data Extraction**

In [6]:
# Data Extraction
candles = pd.read_csv(f"dataset/BTCUSDT-15m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2020-01-01') & (candles['open_time'] < '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_training, 0)
bot._pattern_features_service()

# **Model Training**

In [7]:
# Save data Double Bottom
features = pd.read_csv("features/features_1m.csv")
features = features[features['pattern'] == "Double Bottom"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [8]:
# Parameter Hypertuning Double Bottom
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [9]:
# Model Training Double Bottom
hyper = {'base_score': 7, 'colsample_bylevel': 0, 'colsample_bynode': 4, 'colsample_bytree': 0, 'gamma': 1, 'learning_rate': 6, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 2, 'reg_alpha': 8, 'reg_lambda': 5, 'scale_pos_weight': 9, 'subsample': 3}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DBmodel") 

In [10]:
# Save data Double Top
features = pd.read_csv("features/features_1m.csv")
features = features[features['pattern'] == "Double Top"].reset_index(drop=True)
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.split_data(features)
print("Writing data")
dp.save_data(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [11]:
# Parameter Hypertuning Double Top
X_train, y_train, X_valid, y_valid, X_test, y_test = dp.load_data()
mt.find_hyper_params(X_train, y_train, X_valid, y_valid)

In [12]:
# Model Training Double Top
hyper = {'base_score': 5, 'colsample_bylevel': 0, 'colsample_bynode': 5, 'colsample_bytree': 6, 'gamma': 0, 'learning_rate': 9, 'max_depth': 13, 'min_child_weight': 4, 'n_estimators': 2, 'reg_alpha': 6, 'reg_lambda': 2, 'scale_pos_weight': 10, 'subsample': 9}
X_train, y_train, X_valid, y_valid, X_test, y_test  = dp.load_data()
mt.fit_model(X_train, y_train, X_valid, y_valid, X_test, y_test, hyper, "DTmodel") 

# **Forward Testing**

In [23]:
# Model testing
candles = pd.read_csv(f"dataset/BTCUSDT-1m-2020_2024.csv", index_col=0)
candles_training = candles[(candles['open_time'] >= '2023-01-01')].reset_index(drop=True)

bot = Bot(candles_training, 1)
bot._pattern_features_service()

KeyboardInterrupt: 